# تنظیم دقیق مدل‌های Open AI

این دفترچه بر اساس راهنمای فعلی ارائه شده در [تنظیم دقیق](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) از Open AI نوشته شده است.

تنظیم دقیق باعث بهبود عملکرد مدل‌های پایه برای برنامه شما می‌شود، زیرا مدل را با داده‌ها و زمینه‌های اضافی مرتبط با آن کاربرد یا سناریوی خاص دوباره آموزش می‌دهد. توجه داشته باشید که تکنیک‌هایی مانند _یادگیری با نمونه‌های کم_ و _تولید تقویت‌شده با بازیابی_ به شما این امکان را می‌دهند که درخواست اولیه را با داده‌های مرتبط تقویت کنید تا کیفیت را افزایش دهید. با این حال، این روش‌ها به اندازه حداکثر پنجره توکن مدل پایه هدف محدود هستند.

با تنظیم دقیق، در واقع خود مدل را با داده‌های مورد نیاز دوباره آموزش می‌دهیم (که به ما اجازه می‌دهد مثال‌های بسیار بیشتری نسبت به ظرفیت پنجره توکن استفاده کنیم) و یک نسخه _سفارشی_ از مدل را ارائه می‌دهیم که دیگر نیازی به ارائه مثال‌ها در زمان استنتاج ندارد. این کار نه تنها اثربخشی طراحی درخواست ما را افزایش می‌دهد (در استفاده از پنجره توکن برای موارد دیگر انعطاف بیشتری داریم)، بلکه می‌تواند هزینه‌های ما را نیز کاهش دهد (با کم کردن تعداد توکن‌هایی که باید در زمان استنتاج به مدل ارسال کنیم).

تنظیم دقیق شامل ۴ مرحله است:
1. آماده‌سازی داده‌های آموزشی و بارگذاری آن‌ها.
1. اجرای کار آموزش برای دریافت یک مدل تنظیم‌شده.
1. ارزیابی مدل تنظیم‌شده و تکرار برای بهبود کیفیت.
1. استقرار مدل تنظیم‌شده برای استنتاج پس از رضایت از کیفیت.

توجه داشته باشید که همه مدل‌های پایه از تنظیم دقیق پشتیبانی نمی‌کنند - [مستندات OpenAI را بررسی کنید](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) تا از آخرین اطلاعات مطلع شوید. همچنین می‌توانید یک مدل که قبلاً تنظیم شده را دوباره تنظیم کنید. در این آموزش، ما از `gpt-35-turbo` به عنوان مدل پایه هدف برای تنظیم دقیق استفاده خواهیم کرد.

---


### گام ۱.۱: آماده‌سازی دیتاست

بیایید یک چت‌بات بسازیم که به شما کمک کند جدول تناوبی عناصر را بهتر بفهمید و به سوالات شما درباره هر عنصر با یک شعر لیمریک پاسخ دهد. در این آموزش ساده، فقط یک دیتاست می‌سازیم تا مدل را با چند نمونه پاسخ که فرمت مورد انتظار داده را نشان می‌دهند، آموزش دهیم. در یک کاربرد واقعی، باید دیتاستی با نمونه‌های بسیار بیشتری تهیه کنید. همچنین ممکن است بتوانید از یک دیتاست باز (در حوزه کاربرد خودتان) استفاده کنید و آن را برای استفاده در ریزتنظیم، بازفرمت کنید.

از آنجا که تمرکز ما روی `gpt-35-turbo` است و به دنبال پاسخ تک‌مرحله‌ای (تکمیل چت) هستیم، می‌توانیم نمونه‌ها را با استفاده از [این فرمت پیشنهادی](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) که نیازمندی‌های تکمیل چت OpenAI را منعکس می‌کند، بسازیم. اگر انتظار محتوای مکالمه چندمرحله‌ای را دارید، باید از [فرمت نمونه چندمرحله‌ای](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) استفاده کنید که شامل پارامتر `weight` است تا مشخص کند کدام پیام‌ها باید (یا نباید) در فرآیند ریزتنظیم استفاده شوند.

در این آموزش، از فرمت ساده‌تر تک‌مرحله‌ای استفاده می‌کنیم. داده‌ها در [فرمت jsonl](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) هستند و هر خط یک رکورد دارد که به صورت یک شیء JSON فرمت شده است. قطعه کد زیر دو رکورد نمونه را نشان می‌دهد — برای مجموعه کامل نمونه‌ها (۱۰ مثال) که در آموزش ریزتنظیم استفاده می‌کنیم، به [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) مراجعه کنید. **توجه:** هر رکورد _باید_ در یک خط تعریف شود (و نباید مانند فایل JSON قالب‌بندی‌شده، در چند خط شکسته شود).

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

در یک کاربرد واقعی، برای گرفتن نتایج خوب به مجموعه نمونه‌های بسیار بزرگ‌تری نیاز دارید — اینجا باید بین کیفیت پاسخ‌ها و زمان/هزینه ریزتنظیم تعادل برقرار کنید. ما از یک مجموعه کوچک استفاده می‌کنیم تا بتوانیم فرآیند ریزتنظیم را سریع انجام دهیم و مراحل را نشان دهیم. برای آموزش ریزتنظیم پیچیده‌تر، به [این مثال OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) مراجعه کنید.


### مرحله ۱.۲ بارگذاری دیتاست شما

داده‌ها را با استفاده از Files API [طبق توضیحات اینجا](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file) بارگذاری کنید. توجه داشته باشید که برای اجرای این کد، باید ابتدا مراحل زیر را انجام داده باشید:
 - پکیج پایتون `openai` را نصب کرده باشید (اطمینان حاصل کنید که نسخه شما >=0.28.0 باشد تا از آخرین قابلیت‌ها بهره‌مند شوید)
 - متغیر محیطی `OPENAI_API_KEY` را با کلید API خود از OpenAI تنظیم کرده باشید
برای اطلاعات بیشتر، به [راهنمای راه‌اندازی](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) ارائه‌شده برای این دوره مراجعه کنید.

حالا کد را اجرا کنید تا یک فایل برای بارگذاری از فایل JSONL محلی خود بسازید.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### مرحله ۲.۱: ایجاد شغل تنظیم دقیق با استفاده از SDK



In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### گام ۲.۲: بررسی وضعیت شغل

در اینجا چند کاری که می‌توانید با API مربوط به `client.fine_tuning.jobs` انجام دهید آورده شده است:
- `client.fine_tuning.jobs.list(limit=<n>)` - نمایش آخرین n شغل تنظیم دقیق
- `client.fine_tuning.jobs.retrieve(<job_id>)` - دریافت جزئیات یک شغل تنظیم دقیق خاص
- `client.fine_tuning.jobs.cancel(<job_id>)` - لغو یک شغل تنظیم دقیق
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - نمایش تا n رویداد از شغل مورد نظر
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

اولین مرحله این فرایند _اعتبارسنجی فایل آموزش_ است تا مطمئن شوید داده‌ها در قالب صحیح قرار دارند.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### مرحله ۲.۳: رویدادها را برای پیگیری پیشرفت ثبت کنید


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### مرحله ۲.۴: مشاهده وضعیت در داشبورد OpenAI


شما همچنین می‌توانید وضعیت را با مراجعه به وب‌سایت OpenAI و بررسی بخش _Fine-tuning_ در پلتفرم مشاهده کنید. این بخش به شما وضعیت کار فعلی را نشان می‌دهد و همچنین امکان پیگیری تاریخچه اجرای کارهای قبلی را فراهم می‌کند. در این تصویر، می‌بینید که اجرای قبلی ناموفق بوده و اجرای دوم با موفقیت انجام شده است. برای توضیح بیشتر، این اتفاق زمانی رخ داد که اجرای اول از یک فایل JSON با رکوردهای فرمت‌شده اشتباه استفاده کرد - پس از اصلاح، اجرای دوم با موفقیت به پایان رسید و مدل برای استفاده در دسترس قرار گرفت.

![وضعیت کار Fine-tuning](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.fa.png)


شما همچنین می‌توانید پیام‌های وضعیت و معیارها را با اسکرول کردن به پایین‌تر در داشبورد بصری همان‌طور که در تصویر زیر نشان داده شده مشاهده کنید:

| پیام‌ها | معیارها |
|:---|:---|
| ![Messages](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.fa.png) |  ![Metrics](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.fa.png)|


### مرحله ۳.۱: دریافت شناسه و تست مدل تنظیم‌شده در کد



In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### مرحله ۳.۲: بارگذاری و تست مدل تنظیم‌شده در Playground

حالا می‌توانید مدل تنظیم‌شده خود را به دو روش تست کنید. اول، می‌توانید به Playground بروید و از منوی مدل‌ها، مدل تنظیم‌شده جدیدتان را از میان گزینه‌های موجود انتخاب کنید. روش دیگر این است که از گزینه "Playground" که در پنل تنظیم مدل نمایش داده می‌شود (تصویر بالا را ببینید) استفاده کنید. این گزینه نمایی _مقایسه‌ای_ را اجرا می‌کند که نسخه پایه و نسخه تنظیم‌شده مدل را کنار هم نشان می‌دهد تا بتوانید سریعاً آن‌ها را ارزیابی کنید.

![وضعیت کار تنظیم مدل](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.fa.png)

کافی است همان context سیستمی که در داده‌های آموزشی استفاده کردید را وارد کنید و سوال تستی خود را بنویسید. مشاهده خواهید کرد که هر دو طرف با همان context و سوال به‌روزرسانی می‌شوند. مقایسه را اجرا کنید و تفاوت خروجی‌ها را ببینید. _دقت کنید که مدل تنظیم‌شده پاسخ را دقیقاً با همان فرمتی که در مثال‌هایتان ارائه دادید نمایش می‌دهد، در حالی که مدل پایه فقط از دستور سیستم پیروی می‌کند._

![وضعیت کار تنظیم مدل](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.fa.png)

متوجه خواهید شد که این مقایسه تعداد توکن‌های هر مدل و زمان صرف‌شده برای استنتاج را هم نشان می‌دهد. **این مثال خاص، یک نمونه ساده است که فقط روند کار را نمایش می‌دهد و واقعاً منعکس‌کننده یک دیتاست یا سناریوی واقعی نیست.** ممکن است ببینید که هر دو نمونه تعداد توکن یکسانی دارند (context سیستم و سوال کاربر یکی است) اما مدل تنظیم‌شده زمان بیشتری برای استنتاج صرف می‌کند (مدل سفارشی).

در سناریوهای واقعی، از چنین مثال ساده‌ای استفاده نمی‌کنید، بلکه مدل را با داده‌های واقعی (مثلاً کاتالوگ محصولات برای پشتیبانی مشتری) تنظیم می‌کنید که کیفیت پاسخ‌ها بسیار ملموس‌تر خواهد بود. در _آن_ شرایط، برای رسیدن به کیفیت پاسخ مشابه با مدل پایه، نیاز به مهندسی بیشتر در طراحی prompt خواهید داشت که باعث افزایش مصرف توکن و احتمالاً زمان پردازش استنتاج می‌شود. _برای امتحان این موضوع، نمونه‌های تنظیم مدل را در OpenAI Cookbook ببینید و شروع کنید._



---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما برای دقت تلاش می‌کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطا یا نادقتی باشند. نسخه اصلی سند به زبان مادری آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، ترجمه حرفه‌ای انسانی توصیه می‌شود. ما هیچ مسئولیتی در قبال سوءتفاهم یا تفسیر نادرست ناشی از استفاده از این ترجمه نداریم.
